![Flujo de Data Profiling](asset/eda_pre_title_dataprofiling.png)

El **Data Profiling** tiene como objetivo comprender en profundidad un dataset en su estado inicial (**RAW / PRE**), antes de aplicar transformaciones, modelado o analítica avanzada.

En este proyecto, el proceso se aplica sobre el archivo **`dirty_cafe_sales.csv`**, como parte del proyecto:

**📦 CoffeeSales – Data Profiling (RAW PRE)**

A través de este análisis buscamos:

- Comprender la **estructura real** del dataset recibido.
- Identificar **problemas de calidad de datos** (nulos, duplicados, valores inválidos).
- Evaluar la **consistencia de tipos y formatos**.
- Detectar posibles **reglas de negocio incumplidas**.
- Determinar si el dataset está en condiciones de avanzar a etapas posteriores del pipeline (**decisión GO / NO-GO**).

Este proceso se ejecuta sobre la capa **RAW PRE**, sin aplicar transformaciones, con el fin de obtener un diagnóstico fiel del estado original de los datos.

---

## 🔍 Consultas utilizadas en la exploración (Data Profiling – RAW PRE)

Dataset analizado: **`dirty_cafe_sales.csv`**
Proyecto: **Data Profiling – Coffee Sales**

---

### 1. Vista inicial del dataset
Permite inspeccionar las primeras filas para comprender la estructura general, tipos aparentes de datos y valores representativos.

---

### 2. Estructura de la tabla
Analiza columnas y tipos de datos inferidos para detectar inconsistencias estructurales tempranas.

---

### 3. Conteo de nulos y pseudo-nulos
Identifica valores `NULL`, vacíos y pseudo-nulos (`NaN`, `UNKNOWN`, `ERROR`) para evaluar la completitud del dataset.

---

### 4. Longitudes de texto y numéricos
Evalúa la longitud mínima, máxima y promedio de campos textuales y numéricos (casteados a texto) para detectar valores atípicos o problemas de normalización.

---

### 5. Validación de formato de fechas
Verifica la capacidad de parseo de columnas de fecha, identificando registros con formatos inválidos o no convertibles.

---

### 6. Duplicados por columna
Detecta valores repetidos dentro de cada columna de forma individual para evaluar unicidad y cardinalidad.

---

### 7. Duplicados en múltiples columnas
Analiza duplicados considerando múltiples columnas, permitiendo distinguir duplicados esperados de posibles problemas de integridad.

---

### 8. Validación de reglas de negocio (FACT)
Aplica reglas aritméticas y lógicas a nivel de fila
(ej.: `Quantity × Price Per Unit = Total Spent`)
para detectar errores numéricos, valores inválidos y fallas de consistencia.

---

### 9. Resumen de calidad para columnas DIM
Evalúa columnas categóricas (DIM) clasificando valores en `OK`, `NULL_VALUE` e `INVALID_VALUE`, con métricas porcentuales de impacto.

---

### 10. Resumen de cumplimiento y decisión Go / No-Go
Consolida los resultados de calidad y reglas de negocio para apoyar la decisión de avanzar (**Go**), avanzar con riesgo (**Go con observaciones**) o detener el flujo (**No-Go**).

---

## 🧠 Resultado esperado

El resultado de este proceso es un **diagnóstico técnico documentado** del dataset **`dirty_cafe_sales.csv`**, que permite:

- Justificar una decisión **GO / NO-GO**.
- Definir las transformaciones necesarias para etapas posteriores.
- Sentar la base técnica para **modelado, ETL y arquitectura Medallion**.

![Flujo de Data Profiling](asset/01_vista_inicial.png)

In [1]:
%%sql
-- ========================================
-- EDA_SAMPLE_ROWS
-- Descripción: Ver una muestra de filas para entender el contenido
-- ========================================

SELECT *
FROM profiling.data_profiling_summary
LIMIT 10;

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4,1.0,ERROR,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2,5.0,10.0,UNKNOWN,UNKNOWN,2023-04-27
4,TXN_3160411,Coffee,2,2.0,4.0,Digital Wallet,In-store,2023-06-11
5,TXN_2602893,Smoothie,5,4.0,20.0,Credit Card,NaN,2023-03-31
6,TXN_4433211,UNKNOWN,3,3.0,9.0,ERROR,Takeaway,2023-10-06
7,TXN_6699534,Sandwich,4,4.0,16.0,Cash,UNKNOWN,2023-10-28
8,TXN_4717867,NaN,5,3.0,15.0,NaN,Takeaway,2023-07-28
9,TXN_2064365,Sandwich,5,4.0,20.0,NaN,In-store,2023-12-31


![Flujo de Data Profiling](asset/02_estructura_tabla.png)

In [2]:
%%sql
-- ========================================
-- TEMPLATE: RAW_DESCRIBE_TABLE
-- Descripción: Mostrar estructura (columnas y tipos) de una tabla
-- ========================================

DESCRIBE profiling.data_profiling_summary;

,column_name,column_type,null,key,default,extra
0,Transaction ID,VARCHAR,YES,NaN,NaN,NaN
1,Item,VARCHAR,YES,NaN,NaN,NaN
2,Quantity,VARCHAR,YES,NaN,NaN,NaN
3,Price Per Unit,VARCHAR,YES,NaN,NaN,NaN
4,Total Spent,VARCHAR,YES,NaN,NaN,NaN
5,Payment Method,VARCHAR,YES,NaN,NaN,NaN
6,Location,VARCHAR,YES,NaN,NaN,NaN
7,Transaction Date,VARCHAR,YES,NaN,NaN,NaN


![Flujo de Data Profiling](asset/03_contar_nulos_y_pseudo.png)

In [3]:
%%sql
-- ========================================
-- TEMPLATE: EDA_NULL_PSEUDONULL_COUNT
-- Descripción: Contar NULL reales y pseudo-NULL ('', 'NA', 'NaN', 'UNKNOWN', 'ERROR') por columna
-- ========================================

SELECT
    -- Columna 1
    SUM(CASE
            WHEN "Transaction ID" IS NULL
              OR TRIM("Transaction ID") IN ('', 'NA', 'NaN', 'UNKNOWN', 'ERROR')
        THEN 1 ELSE 0 END) AS Transaction_ID_null_like,

    -- Columna 2
    SUM(CASE
            WHEN Item IS NULL
              OR TRIM(Item) IN ('', 'NA', 'NaN', 'UNKNOWN', 'ERROR')
        THEN 1 ELSE 0 END) AS Item_null_like,

    -- Columna 3
    SUM(CASE
            WHEN Quantity IS NULL
              OR TRIM(Quantity) IN ('', 'NA', 'NaN', 'UNKNOWN', 'ERROR')
        THEN 1 ELSE 0 END) AS Quantity_null_like,

    -- Columna 4
    SUM(CASE
            WHEN "Price Per Unit" IS NULL
              OR TRIM("Price Per Unit") IN ('', 'NA', 'NaN', 'UNKNOWN', 'ERROR')
        THEN 1 ELSE 0 END) AS Price_Per_Unit_null_like,

    -- Columna 5
    SUM(CASE
            WHEN "Total Spent" IS NULL
              OR TRIM("Total Spent") IN ('', 'NA', 'NaN', 'UNKNOWN', 'ERROR')
        THEN 1 ELSE 0 END) AS Total_Spent_null_like,

    -- Columna 6
    SUM(CASE
            WHEN "Payment Method" IS NULL
              OR TRIM("Payment Method") IN ('', 'NA', 'NaN', 'UNKNOWN', 'ERROR')
        THEN 1 ELSE 0 END) AS Payment_Method_null_like,

    -- Columna 7
    SUM(CASE
            WHEN Location IS NULL
              OR TRIM(Location) IN ('', 'NA', 'NaN', 'UNKNOWN', 'ERROR')
        THEN 1 ELSE 0 END) AS Location_null_like,

    -- Columna 8
    SUM(CASE
            WHEN "Transaction Date" IS NULL
              OR TRIM("Transaction Date") IN ('', 'NA', 'NaN', 'UNKNOWN', 'ERROR')
        THEN 1 ELSE 0 END) AS Transaction_Date_null_like

FROM profiling.data_profiling_summary;

,Transaction_ID_null_like,Item_null_like,Quantity_null_like,Price_Per_Unit_null_like,Total_Spent_null_like,Payment_Method_null_like,Location_null_like,Transaction_Date_null_like
0,0,969,479,533,502,3178,3961,460


![Flujo de Data Profiling](asset/04_longitudes_texto.png)

In [1]:
%%sql
-- ========================================
-- TEMPLATE: EDA_TEXT_LENGTH_PROFILE_MULTI
-- Descripción: Perfil de longitudes (texto o numérico casteado) por múltiples columnas
-- ========================================

SELECT
    columna,
    MIN(len) AS min_length,
    MAX(len) AS max_length,
    ROUND(AVG(len), 2) AS avg_length,
    SUM(CASE WHEN len = 0 THEN 1 ELSE 0 END) AS empty_values
FROM (
    -- Columna 1 (texto)
    SELECT
        'Item' AS columna,
        LENGTH(COALESCE(CAST(Item AS VARCHAR), '')) AS len
    FROM profiling.data_profiling_summary

    UNION ALL

    -- Columna 2 (numérica casteada)
    SELECT
        'Quantity' AS columna,
        LENGTH(COALESCE(CAST(Quantity AS VARCHAR), '')) AS len
    FROM profiling.data_profiling_summary

    UNION ALL

       -- Columna 2 (numérica casteada)
    SELECT
        'Price Per Unit' AS columna,
        LENGTH(COALESCE(CAST("Price Per Unit" AS VARCHAR), '')) AS len
    FROM profiling.data_profiling_summary

    UNION ALL

    -- Columna 3 (numérica casteada)
    SELECT
        'Total Spent' AS columna,
        LENGTH(COALESCE(CAST("Total Spent" AS VARCHAR), '')) AS len
    FROM profiling.data_profiling_summary

) t
GROUP BY columna
ORDER BY max_length DESC;

,columna,min_length,max_length,avg_length,empty_values
0,Item,0,8,5.46,333
1,Quantity,0,7,1.16,138
2,Price Per Unit,0,7,3.05,179
3,Total Spent,0,7,3.41,173


![Flujo de Data Profiling](asset/05_validar_formato_fecha.png)

In [2]:
%%sql
-- ========================================
-- TEMPLATE: EDA_DATE_FORMAT_VALIDATION
-- Descripción: Validar formato y parseo de columnas de fecha
-- Objetivo: Detectar fechas inválidas o no convertibles
-- ========================================

SELECT
    'Transaction Date' AS columna,
    COUNT(*) AS total_registros,
    COUNT(
        CASE
            WHEN TRY_CAST("Transaction Date" AS DATE) IS NULL
            THEN 1
        END
    ) AS fechas_invalidas
FROM profiling.data_profiling_summary;

,columna,total_registros,fechas_invalidas
0,Transaction Date,10000,460


## Identificar valores de fecha no parseable

In [3]:
%%sql
SELECT
    "Transaction Date",
    COUNT(*) AS ocurrencias
FROM profiling.data_profiling_summary
WHERE TRY_CAST("Transaction Date" AS DATE) IS NULL
GROUP BY "Transaction Date"
ORDER BY ocurrencias DESC;

,Transaction Date,ocurrencias
0,NaN,159
1,UNKNOWN,159
2,ERROR,142


![Flujo de Data Profiling](asset/06_duplicado_por_columnas.png)

In [5]:
%%sql
-- ========================================
-- TEMPLATE: EDA_DUPLICATES_ALL_COLUMNS
-- Descripción: Detectar duplicados por columna completa
-- ========================================

SELECT
    columna,
    COUNT(*) AS total_registros,
    COUNT(DISTINCT valor) AS valores_unicos,
    COUNT(*) - COUNT(DISTINCT valor) AS duplicados
FROM (
    -- Columna 1
    SELECT 'Transaction ID' AS columna, CAST("Transaction ID" AS VARCHAR) AS valor
    FROM profiling.data_profiling_summary

    UNION ALL
    -- Columna 2
    SELECT 'Item', CAST("Item" AS VARCHAR)
    FROM profiling.data_profiling_summary

    UNION ALL
    -- Columna 3
    SELECT 'Quantity', CAST(Quantity AS VARCHAR)
    FROM profiling.data_profiling_summary

    UNION ALL
    -- Columna 4
    SELECT 'Price Per Unit', CAST("Price Per Unit" AS VARCHAR)
    FROM profiling.data_profiling_summary

    UNION ALL
    -- Columna 5
    SELECT 'Total Spent', CAST("Total Spent" AS VARCHAR)
    FROM profiling.data_profiling_summary

    UNION ALL
    -- Columna 6
    SELECT 'Payment Method', CAST("Payment Method" AS VARCHAR)
    FROM profiling.data_profiling_summary

    UNION ALL
    -- Columna 7
    SELECT 'Location', CAST(Location AS VARCHAR)
    FROM profiling.data_profiling_summary

    UNION ALL
    -- Columna 8
    SELECT 'Transaction Date', CAST("Transaction Date" AS VARCHAR)
    FROM profiling.data_profiling_summary
) t
GROUP BY columna
ORDER BY duplicados DESC;

,columna,total_registros,valores_unicos,duplicados
0,Location,10000,4,9996
1,Payment Method,10000,5,9995
2,Quantity,10000,7,9993
3,Price Per Unit,10000,8,9992
4,Item,10000,10,9990
5,Total Spent,10000,19,9981
6,Transaction Date,10000,367,9633
7,Transaction ID,10000,10000,0


![Flujo de Data Profiling](asset/07_duplicados_multiple_columnas.png)

In [6]:
%%sql
-- ========================================
-- TEMPLATE: EDA_DUPLICATES_MULTI_COLUMN
-- Descripción: Detectar valores duplicados en múltiples columnas
-- ========================================

SELECT
    columna,
    valor,
    COUNT(*) AS repeticiones
FROM (
    -- Columna 1
    SELECT 'Location' AS columna, Location AS valor
    FROM profiling.data_profiling_summary

    UNION ALL
    -- Columna 2
    SELECT 'Payment Method', "Payment Method"
    FROM profiling.data_profiling_summary

    UNION ALL
    -- Columna 3
    SELECT 'Quantity', Quantity
    FROM profiling.data_profiling_summary

    UNION ALL
    -- Columna 4
    SELECT 'Price Per Unit', "Price Per Unit"
    FROM profiling.data_profiling_summary

     UNION ALL
    -- Columna 5
    SELECT 'Item', Item
    FROM profiling.data_profiling_summary

    UNION ALL
    -- Columna 6
    SELECT 'Total Spent', "Total Spent"
    FROM profiling.data_profiling_summary

    UNION ALL
    -- Columna 7
    SELECT 'Transaction Date', "Transaction Date"
    FROM profiling.data_profiling_summary
) t
GROUP BY columna, valor
HAVING COUNT(*) > 1
ORDER BY columna, repeticiones DESC;

,columna,valor,repeticiones
0,Item,Juice,1171
1,Item,Coffee,1165
2,Item,Salad,1148
3,Item,Cake,1139
4,Item,Sandwich,1131
...,...,...,...
422,Transaction Date,2023-07-30,15
423,Transaction Date,2023-11-24,15
424,Transaction Date,2023-03-11,14
425,Transaction Date,2023-02-17,14


![Flujo de Data Profiling](asset/08_validar_reglas_negocios.png)

In [7]:
%%sql
-- ========================================
-- TEMPLATE: EDA_VALIDATE_BUSINESS_RULES
-- Descripción: Validar regla de negocio fila a fila
-- Regla ejemplo: Quantity * Price Per Unit = Total Spent
-- ========================================

SELECT
    *,
    CASE
        WHEN Quantity    IN ('NaN','UNKNOWN','ERROR')
          OR "Price Per Unit"  IN ('NaN','UNKNOWN','ERROR')
          OR "Total Spent"  IN ('NaN','UNKNOWN','ERROR')
        THEN 'INVALID_VALUE'

        WHEN TRY_CAST(Quantity AS DOUBLE) IS NULL
          OR TRY_CAST("Price Per Unit" AS DOUBLE) IS NULL
          OR TRY_CAST("Total Spent" AS DOUBLE) IS NULL
        THEN 'CAST_FAIL'

        WHEN TRY_CAST(Quantity AS DOUBLE)
             * TRY_CAST("Price Per Unit" AS DOUBLE)
             = TRY_CAST("Total Spent" AS DOUBLE)
        THEN 'OK'
        ELSE 'BUSINESS_ERROR'
    END AS check_total
FROM profiling.data_profiling_summary
LIMIT 10;

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date,check_total
0,TXN_1961373,Coffee,2,2.0,4.0,Credit Card,Takeaway,2023-09-08,OK
1,TXN_4977031,Cake,4,3.0,12.0,Cash,In-store,2023-05-16,OK
2,TXN_4271903,Cookie,4,1.0,ERROR,Credit Card,In-store,2023-07-19,INVALID_VALUE
3,TXN_7034554,Salad,2,5.0,10.0,UNKNOWN,UNKNOWN,2023-04-27,OK
4,TXN_3160411,Coffee,2,2.0,4.0,Digital Wallet,In-store,2023-06-11,OK
5,TXN_2602893,Smoothie,5,4.0,20.0,Credit Card,NaN,2023-03-31,OK
6,TXN_4433211,UNKNOWN,3,3.0,9.0,ERROR,Takeaway,2023-10-06,OK
7,TXN_6699534,Sandwich,4,4.0,16.0,Cash,UNKNOWN,2023-10-28,OK
8,TXN_4717867,NaN,5,3.0,15.0,NaN,Takeaway,2023-07-28,OK
9,TXN_2064365,Sandwich,5,4.0,20.0,NaN,In-store,2023-12-31,OK


![Flujo de Data Profiling](asset/09_resumen_calidad_DIM.png)

In [4]:
%%sql
-- ========================================
-- TEMPLATE: EDA_VALIDATE_DIM_QUALITY_SUMMARY
-- Descripción: Resumen de calidad para columnas DIM
-- ========================================

WITH base AS (
    SELECT
        columna,
        CASE
            WHEN valor IS NULL
              OR TRIM(valor) = ''
            THEN 'NULL_VALUE'

            WHEN valor IN ('NaN','UNKNOWN','ERROR')
            THEN 'INVALID_VALUE'

            ELSE 'OK'
        END AS status
    FROM (
        -- DIM 1
        SELECT 'Item' AS columna, CAST(Item AS VARCHAR) AS valor
        FROM profiling.data_profiling_summary

        UNION ALL
        -- DIM 2
        SELECT 'Payment Method', CAST("Payment Method" AS VARCHAR)
        FROM profiling.data_profiling_summary

        UNION ALL
        -- DIM 3
        SELECT 'Location', CAST(Location AS VARCHAR)
        FROM profiling.data_profiling_summary
    ) t
),
stats AS (
    SELECT
        columna,
        status,
        COUNT(*) AS total
    FROM base
    GROUP BY columna, status
),
tot AS (
    SELECT
        columna,
        SUM(total) AS grand_total
    FROM stats
    GROUP BY columna
)
SELECT
    s.columna,
    s.status,
    s.total,
    ROUND(100.0 * s.total / t.grand_total, 2) AS porcentaje
FROM stats s
JOIN tot t ON s.columna = t.columna
ORDER BY s.columna, porcentaje DESC;

,columna,status,total,porcentaje
0,Item,OK,9031,90.31
1,Item,INVALID_VALUE,636,6.36
2,Item,NULL_VALUE,333,3.33
3,Location,OK,6039,60.39
4,Location,NULL_VALUE,3265,32.65
5,Location,INVALID_VALUE,696,6.96
6,Payment Method,OK,6822,68.22
7,Payment Method,NULL_VALUE,2579,25.79
8,Payment Method,INVALID_VALUE,599,5.99


![Flujo de Data Profiling](asset/10_resumen_calidad_FACT.png)

In [1]:
%%sql
-- ========================================
-- TEMPLATE: EDA_VALIDATE_BUSINESS_RULES_SUMMARY
-- Descripción: Resumen de cumplimiento de regla de negocio
-- Regla ejemplo: Quantity * Price Per Unit = Total Spent
-- ========================================

WITH base AS (
    SELECT
        CASE
            WHEN Quantity    IN ('NaN','UNKNOWN','ERROR')
              OR "Price Per Unit"  IN ('NaN','UNKNOWN','ERROR')
              OR "Total Spent"  IN ('NaN','UNKNOWN','ERROR')
            THEN 'INVALID_VALUE'

            WHEN TRY_CAST(Quantity AS DOUBLE) IS NULL
              OR TRY_CAST("Price Per Unit" AS DOUBLE) IS NULL
              OR TRY_CAST("Total Spent" AS DOUBLE) IS NULL
            THEN 'CAST_FAIL'

            WHEN TRY_CAST(Quantity AS DOUBLE)
                 * TRY_CAST("Price Per Unit" AS DOUBLE)
                 = TRY_CAST("Total Spent" AS DOUBLE)
            THEN 'OK'
            ELSE 'BUSINESS_ERROR'
        END AS rule_status
    FROM profiling.data_profiling_summary
),
stats AS (
    SELECT
        rule_status,
        COUNT(*) AS total
    FROM base
    GROUP BY rule_status
),
tot AS (
    SELECT SUM(total) AS grand_total FROM stats
)
SELECT
    s.rule_status,
    s.total,
    ROUND(100.0 * s.total / t.grand_total, 2) AS porcentaje
FROM stats s
CROSS JOIN tot t
ORDER BY s.rule_status;

,rule_status,total,porcentaje
0,CAST_FAIL,461,4.61
1,INVALID_VALUE,995,9.95
2,OK,8544,85.44


![Flujo de Data Profiling](asset/FINAL_validacion_go_y_no_go.png)

In [8]:
%%sql
-- ========================================
-- TEMPLATE: EDA_FINAL_SCORE_FACT_DIM
-- Descripción: Score final consolidado (FACT + DIM) para apoyar decisión Go/No-Go
-- Motor: DuckDB
-- ========================================

WITH
-- ----------------------------
-- Parámetros / umbrales (ajusta aquí)
-- ----------------------------
params AS (
    SELECT
        90.0 AS thr_fact_ok_pct,  -- % mínimo OK en FACT
        80.0 AS thr_dim_ok_pct,   -- % mínimo OK en DIM
        5.0  AS thr_date_invalid_pct -- % máximo fechas inválidas (si lo agregas luego)
),

-- ----------------------------
-- FACT: Regla de negocio (ejemplo)
-- Quantity * Price Per Unit = Total Spent
-- ----------------------------
fact_base AS (
    SELECT
        CASE
            WHEN Quantity IN ('NaN','UNKNOWN','ERROR')
              OR "Price Per Unit" IN ('NaN','UNKNOWN','ERROR')
              OR "Total Spent" IN ('NaN','UNKNOWN','ERROR')
            THEN 'INVALID_VALUE'

            WHEN TRY_CAST(Quantity AS DOUBLE) IS NULL
              OR TRY_CAST("Price Per Unit" AS DOUBLE) IS NULL
              OR TRY_CAST("Total Spent" AS DOUBLE) IS NULL
            THEN 'CAST_FAIL'

            WHEN TRY_CAST(Quantity AS DOUBLE)
               * TRY_CAST("Price Per Unit" AS DOUBLE)
               = TRY_CAST("Total Spent" AS DOUBLE)
            THEN 'OK'
            ELSE 'BUSINESS_ERROR'
        END AS status
    FROM profiling.data_profiling_summary
),
fact_stats AS (
    SELECT
        'FACT_RULE_TOTAL_SPENT' AS check_name,
        COUNT(*) AS total_rows,
        SUM(CASE WHEN status = 'OK' THEN 1 ELSE 0 END) AS ok_rows,
        SUM(CASE WHEN status <> 'OK' THEN 1 ELSE 0 END) AS bad_rows
    FROM fact_base
),
fact_score AS (
    SELECT
        check_name,
        total_rows,
        ok_rows,
        bad_rows,
        ROUND(100.0 * ok_rows / NULLIF(total_rows,0), 2) AS ok_pct
    FROM fact_stats
),

-- ----------------------------
-- DIM: Calidad por columnas (NULL / inválidos)
-- Agrega/quita columnas DIM en el UNION ALL
-- ----------------------------
dim_base AS (
    SELECT
        columna,
        CASE
            WHEN valor IS NULL OR TRIM(valor) = '' THEN 'NULL_VALUE'
            WHEN valor IN ('NaN','UNKNOWN','ERROR') THEN 'INVALID_VALUE'
            ELSE 'OK'
        END AS status
    FROM (
        SELECT 'Item' AS columna, CAST(Item AS VARCHAR) AS valor FROM profiling.data_profiling_summary
        UNION ALL
        SELECT 'Payment Method', CAST("Payment Method" AS VARCHAR) FROM profiling.data_profiling_summary
        UNION ALL
        SELECT 'Location', CAST(Location AS VARCHAR) FROM profiling.data_profiling_summary
    ) t
),
dim_stats AS (
    SELECT
        'DIM_QUALITY' AS check_name,
        COUNT(*) AS total_cells,
        SUM(CASE WHEN status = 'OK' THEN 1 ELSE 0 END) AS ok_cells,
        SUM(CASE WHEN status <> 'OK' THEN 1 ELSE 0 END) AS bad_cells
    FROM dim_base
),
dim_score AS (
    SELECT
        check_name,
        total_cells AS total_rows,
        ok_cells AS ok_rows,
        bad_cells AS bad_rows,
        ROUND(100.0 * ok_cells / NULLIF(total_cells,0), 2) AS ok_pct
    FROM dim_stats
),

-- ----------------------------
-- Consolidado (FACT + DIM)
-- ----------------------------
scores AS (
    SELECT * FROM fact_score
    UNION ALL
    SELECT * FROM dim_score
),

-- ----------------------------
-- Decisión Go/No-Go basada en umbrales
-- ----------------------------
decision AS (
    SELECT
        MAX(CASE WHEN check_name = 'FACT_RULE_TOTAL_SPENT' THEN ok_pct END) AS fact_ok_pct,
        MAX(CASE WHEN check_name = 'DIM_QUALITY' THEN ok_pct END) AS dim_ok_pct
    FROM scores
)

SELECT
    s.check_name,
    s.total_rows,
    s.ok_rows,
    s.bad_rows,
    s.ok_pct,
    CASE
        WHEN s.check_name = 'FACT_RULE_TOTAL_SPENT'
             AND s.ok_pct >= (SELECT thr_fact_ok_pct FROM params)
        THEN 'PASS'
        WHEN s.check_name = 'DIM_QUALITY'
             AND s.ok_pct >= (SELECT thr_dim_ok_pct FROM params)
        THEN 'PASS'
        ELSE 'FAIL'
    END AS threshold_status
FROM scores s
ORDER BY s.check_name;

,check_name,total_rows,ok_rows,bad_rows,ok_pct,threshold_status
0,DIM_QUALITY,30000,21892,8108,72.97,FAIL
1,FACT_RULE_TOTAL_SPENT,10000,8544,1456,85.44,FAIL
